# Extended Kalman Filter localization example
We are considering a robot with differential traction with a rotary sensor on its top. It detects beacons and obtain a measure of its distance and angle to the robot refential frame.
Whenever we observe a beacon there is an error in the estimated angle and an error in estimating the distance. We assume that the standard deviation of the distance error increases proportionally with distance.

This work is inspired on "Laboratorial Work 4 - Robot localization based on beacons" from the course "Sistemas Robóticos Autónomos" at FEUP.
More specifically we are talking about a soccer robot in a small soccer field with the beacons in the four corner. The robot uses first a rotative of a camera and then an omnidirectional camera in the second part. 

## 1. Extended Kalman Filter implementation in robot simulator
We start by importing numpy

In [9]:
import numpy as np

and setting the initial state of the robot at

In [10]:
x=np.array([2, -2, 0])
x

array([ 2, -2,  0])

Differently from MATLAB at this stage does not make sense to distinguish from "horizontal" or "vertical" vectors.

We are going to simulate

In [12]:
N=300

time intervals.

Now we generate the control signals of the robot for each time interval

In [14]:
v_t = np.random.normal(0,1,N) + 1 #linear velocity
omega_t = np.random.normal(0,1,N)*0.5 + 0.5 #angular velocity

and we group them in

In [20]:
u_t = np.array([v_t,omega_t])
u_t.shape

(2, 300)

Covariance...

## 2. Implementation considering only angle or distance measure
Content

## 3. Four beacons simultaneously
Content

## 4. Four beacons: only angle or distance
Content

## 5. Observations in coordinates of the robot referential frame
Content